In [39]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
import unidecode
import pandas as pd
import numpy as np
import time

In [40]:
from utils import check_if_titles_match, lower_and_remove_diacritics, check_row_similariy

Already scraped

artist_name = "Severina"
artist_name = "Mia Dimšić"
artist_name = "Jelena Rozga"
artist_name = "Petar Grašo"
artist_name = "Đani Maršan"
artist_name = "Aerodrom"


In [72]:
artist_name = "Alen Vitasović"


In [73]:
df = pd.read_csv(f"data/{artist_name}_tekstovinet.csv")

df.head()

,Song_ID,Artist,Artist_diskografija,Song_tekstovinet,Views_tekstovinet,Lyrics_tekstovinet,Url_tekstovinet
0,H/8bQC7v_1_0,Alen Vitasović,Alen Vitasović,Ako tebi lazen ja,2417,"Mislim na tebe\r\nka iz spine curi vrime, brzo...","https://tekstovi.net/2,2448,42582.html"
1,H/8bQC7v_1_1,Alen Vitasović,Alen Vitasović,Bura,7517,"Bura, svo je grane slomila\r\ni tvoje je tilo ...","https://tekstovi.net/2,2448,32922.html"
2,H/8bQC7v_1_2,Alen Vitasović,Alen Vitasović,Ca mi sad ne smi rec (feat. Zdenko Hrsak),1705,"Vani je cvika, po krovu ledi tuce \r\nnema kuc...","https://tekstovi.net/2,2448,52349.html"
3,H/8bQC7v_1_3,Alen Vitasović,Alen Vitasović,Cesta priko mora,5463,Ja san s tobon biti stija\r\n i po svitu san h...,"https://tekstovi.net/2,2448,47629.html"
4,H/8bQC7v_1_4,Alen Vitasović,Alen Vitasović,Cvit,1052,Moglo je pasati cuda bolje\r\nvrimena je bilo ...,"https://tekstovi.net/2,2448,58376.html"


In [74]:
df_problematic = pd.DataFrame(columns =  ["Artist"])

In [44]:
def search_google(search_query):
     #search
    driver.find_element(By.NAME, 'q').clear()
    search_box = driver.find_element(By.NAME, 'q')
    search_box.send_keys(search_query)
    search_box.send_keys(Keys.RETURN)
    time.sleep(3)

In [58]:
def get_lyricstranslate():

    driver.find_element(By.PARTIAL_LINK_TEXT, 'lyricstranslate.com').send_keys(Keys.CONTROL + Keys.RETURN)
    time.sleep(0.5)
    window_handles = driver.window_handles
    driver.switch_to.window(window_name=window_handles[1])    
    time.sleep(1)


    try: driver.find_element(By.XPATH, "//*[contains(text(), 'AGREE')]").click()
    except: pass
    time.sleep(0.5)
    
    try: driver.find_element(By.PARTIAL_LINK_TEXT, 'English').click()
    except: pass
    time.sleep(0.5)

    try: driver.find_element(By.PARTIAL_LINK_TEXT, 'Click to see the original lyrics').click()
    except: pass
    time.sleep(0.5)

    
    try:
        original_lyrics_raw = driver.find_element(By.CLASS_NAME, 'song-node-text')
        original_title = original_lyrics_raw.find_element(By.CLASS_NAME, 'title-h2').text
        par_class = original_lyrics_raw.find_elements(By.CLASS_NAME, 'par')
        original_lyrics = ". ".join([t.text for t in par_class])
        original_lyrics = original_lyrics.replace("*", "")
    except: 
        print("Couldn't find english lyrics")
        original_title = np.nan
        original_lyrics = np.nan

    try:
        eng_lyrics_raw = driver.find_element(By.CLASS_NAME, 'translate-node-text')
        eng_title = eng_lyrics_raw.find_element(By.CLASS_NAME, 'title-h2').text
        par_class = eng_lyrics_raw.find_elements(By.CLASS_NAME, 'par')
        eng_lyrics = ". ".join([t.text for t in par_class])
        eng_lyrics = eng_lyrics.replace("*", "")
    except: 
        print("Couldn't find english lyrics")
        eng_title = np.nan
        eng_lyrics = np.nan

        
    try:
        copyright = original_lyrics_raw.find_element(By.CLASS_NAME, 'copyrighttext').text.split("\n")
        writer_info = [c for c in copyright if "Writer" in c][0]
        if len(writer_info)>0:
            writer = writer_info.split(": ")[1]

    except: 
        print("Couldn't find writer info")
        writer = np.nan


    try:
        song_info = driver.find_element(By.CLASS_NAME, 'song-node-info')
        album_name = song_info.find_element(By.XPATH, "//*[contains(text(), 'Album')]").text
        album_name = album_name.split(": ")[1]
    except: 
        print("Couldn't find album info")
        album_name = np.nan

    
    song_url = driver.current_url
    driver.close()
    driver.switch_to.window(window_name=window_handles[0])
    
    
    return writer, original_title, eng_title, album_name, original_lyrics, eng_lyrics, song_url

In [75]:
ser = Service("chromedriver.exe")
op = webdriver.ChromeOptions()
driver = webdriver.Chrome(service=ser, options=op)
driver.get("https://google.com")
time.sleep(0.5)
try: driver.find_element(By.XPATH, "//*[contains(text(), 'Aceito')]").click()
except: driver.find_element(By.XPATH, "//*[contains(text(), 'I agree')]").click()
time.sleep(0.5)
try: driver.find_element(By.XPATH, "//*[contains(text(), 'English')]").click()
except: pass
time.sleep(0.5)

df_new = df.copy()#[0:1]##[16:17]
for row in df_new.iterrows():
    time.sleep(1)
    artist_name = row[1].Artist
    song_name = row[1].Song_tekstovinet
    try: song_name = song_name[0:song_name.index("(")]
    except:pass
    
    try:
        search_google(artist_name+" "+song_name+ " english lyricstranslate")
        writer, original_title, eng_title, album_name, original_lyrics, eng_lyrics, song_url = get_lyricstranslate()

        df.loc[row[0], "Writer_lyricstranslate"] = writer 
        df.loc[row[0], "Song_lyricstranslate"] = original_title
        df.loc[row[0], "EngTitle_lyricstranslate"] = eng_title
        df.loc[row[0], "Album_lyricstranslate"] = album_name
        df.loc[row[0], "Lyrics_lyricstranslate"] = original_lyrics
        df.loc[row[0], "EngLyrics_lyricstranslate"] = eng_lyrics
        df.loc[row[0], "Url_lyricstranslate"] = song_url

        print(f"Succesfully found information for {song_name} from {artist_name}")
    
    except Exception as e:
        df_problematic.loc[row[0], "Song_ID"] = row[1].Song_ID
        df_problematic.loc[row[0], "Artist"] = artist_name
        df_problematic.loc[row[0], "Song_lyricstranslate_run"] = song_name
        df_problematic.loc[row[0], "Error_lyricstranslate"] = e
        print(f"Problem with finding info for {song_name} from {artist_name}")
        print(e)
    print("#######")
    time.sleep(3)
    


Couldn't find english lyrics
Couldn't find writer info
Couldn't find album info
Succesfully found information for Ako tebi lazen ja from Alen Vitasović
#######
Couldn't find english lyrics
Couldn't find writer info
Couldn't find album info
Succesfully found information for Bura from Alen Vitasović
#######
Problem with finding info for Ca mi sad ne smi rec  from Alen Vitasović
Message: no such element: Unable to locate element: {"method":"partial link text","selector":"lyricstranslate.com"}
  (Session info: chrome=98.0.4758.102)
Stacktrace:
Backtrace:
	Ordinal0 [0x002B69A3+2582947]
	Ordinal0 [0x0024A6D1+2139857]
	Ordinal0 [0x00143A98+1063576]
	Ordinal0 [0x0016FF3E+1244990]
	Ordinal0 [0x0017013B+1245499]
	Ordinal0 [0x00199F52+1417042]
	Ordinal0 [0x00188594+1344916]
	Ordinal0 [0x0019834A+1409866]
	Ordinal0 [0x00188366+1344358]
	Ordinal0 [0x00165176+1200502]
	Ordinal0 [0x00166066+1204326]
	GetHandleVerifier [0x0045BE02+1675858]
	GetHandleVerifier [0x0051036C+2414524]
	GetHandleVerifier [0x

In [76]:
#check if there are some very short lyrics
df.Lyrics_lyricstranslate.dropna().map(lambda x: len(x)<10).sum()

0

In [77]:
wrong_rows = check_row_similariy(df, "Lyrics_tekstovinet", "Lyrics_lyricstranslate", threshold=0.7)

moglo je pasati cuda boljevrimena je bilo ma ni bilo voljebija san mlad da na vrime bin kapijada san sve z pocetka krivo ucinijamoglo je pasati cuda lagljeda u magli nis frma da san posa daljeznan da tvoje san srce zlomijama ni jenu kako tebe nis volija pus da buden cvit u tvojen poljudrugaciji ud svih ke ces ubratipus da buden samo vitar kojeg ces udahnutsamo da san s tobon jos jedan putmoglo je pasati cuda hujeca je bilo moje sve bi bilo tujeto ne bis bila ti to ne bin bija jatebe nikad ne bin upoznapus da buden cvit u tvojen poljudrugacij ud svih ke ces ubratipus da buden samo lisce na koje ces se lecinek me tvoje tilo stisce dok nan ruka ruku iscepus da buden dazd u tvojoj arijimagari samo kap ka ce na te pastipus da buden samo vitar kojeg ces udahnutsamo da san s tobon jos jedan put
XXXXXX
ti si kako i cvit tebi nis ne rabija san jeno nis boje bisi zabiti si kako i grojze svaki bi te stijaja bin te pozoba pa bin te pustija bolje da si najdes kega ki ce tvoju mamu kuntentatiki na o

In [78]:
wrong_rows

[4, 8, 12, 19, 22]

In [79]:
unmatched_titles = check_if_titles_match(df, "Song_tekstovinet", "Song_lyricstranslate")

In [80]:
unmatched_titles.loc[[r for r in unmatched_titles.index if r not in wrong_rows]]

,Song_tekstovinet,Song_lyricstranslate
16,ne moren bez nje,ja ne moren bez nje
17,neka se zatare,da se ne zatare


In [81]:
[r for r in  wrong_rows if r not in  unmatched_titles.index]

[]

In [82]:
##set parameters as nan for wrong rows
for row in wrong_rows:
    df.loc[row, "Writer_lyricstranslate"] = np.nan
    df.loc[row, "Song_lyricstranslate"] = np.nan
    df.loc[row, "EngTitle_lyricstranslate"] = np.nan
    df.loc[row, "Album_lyricstranslate"] = np.nan
    df.loc[row, "Lyrics_lyricstranslate"] = np.nan
    df.loc[row, "EngLyrics_lyricstranslate"] = np.nan
    df.loc[row, "Url_lyricstranslate"] = np.nan

In [83]:
#check if number of nans is what you would expect
df.Lyrics_lyricstranslate.isnull().sum()

20

In [84]:
len(df)

27

In [86]:
df[df.Lyrics_lyricstranslate.isnull()]

,Song_ID,Artist,Artist_diskografija,Song_tekstovinet,Views_tekstovinet,Lyrics_tekstovinet,Url_tekstovinet,Writer_lyricstranslate,Song_lyricstranslate,EngTitle_lyricstranslate,Album_lyricstranslate,Lyrics_lyricstranslate,EngLyrics_lyricstranslate,Url_lyricstranslate
2,H/8bQC7v_1_2,Alen Vitasović,Alen Vitasović,Ca mi sad ne smi rec (feat. Zdenko Hrsak),1705,"Vani je cvika, po krovu ledi tuce \r\nnema kuc...","https://tekstovi.net/2,2448,52349.html",NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,H/8bQC7v_1_3,Alen Vitasović,Alen Vitasović,Cesta priko mora,5463,Ja san s tobon biti stija\r\n i po svitu san h...,"https://tekstovi.net/2,2448,47629.html",NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,H/8bQC7v_1_4,Alen Vitasović,Alen Vitasović,Cvit,1052,Moglo je pasati cuda bolje\r\nvrimena je bilo ...,"https://tekstovi.net/2,2448,58376.html",NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,H/8bQC7v_1_5,Alen Vitasović,Alen Vitasović,Dajla,403,Ovdje jutra mirisu na tebe \r\nkao cvijece kad...,"https://tekstovi.net/2,2448,61492.html",NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,H/8bQC7v_1_6,Alen Vitasović,Alen Vitasović,Groznjana grada (duet Scike),1726,Lipa mlada\r\nspod Groznjana grada\r\nreci nan...,"https://tekstovi.net/2,2448,49981.html",NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,H/8bQC7v_1_8,Alen Vitasović,Alen Vitasović,Ja ne gren,5051,Su mi rekli da je kasno i da moraju zaprit\r\n...,"https://tekstovi.net/2,2448,40441.html",NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,H/8bQC7v_1_9,Alen Vitasović,Alen Vitasović,Ja san Istrijan,5575,Ja volin poj u Split\r\npar dani doli cu bit\r...,"https://tekstovi.net/2,2448,39390.html",NaN,NaN,NaN,NaN,NaN,NaN,NaN
11,H/8bQC7v_1_11,Alen Vitasović,Alen Vitasović,Kad ce vrime od kruha i grozja,1877,"Nis drugo ne triba \r\ni nis ne ceka, ajmo vec...","https://tekstovi.net/2,2448,47508.html",NaN,NaN,NaN,NaN,NaN,NaN,NaN
12,H/8bQC7v_1_12,Alen Vitasović,Alen Vitasović,Ko mi reces da,2923,Ca bin za te ucinija\r\nca bin za te da\r\npos...,"https://tekstovi.net/2,2448,40442.html",NaN,NaN,NaN,NaN,NaN,NaN,NaN
13,H/8bQC7v_1_13,Alen Vitasović,Alen Vitasović,Lipe oci,2052,Ona mi je bila\r\nsve ca san prova\r\nma kamo ...,"https://tekstovi.net/2,2448,49982.html",NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [69]:
df.Lyrics_lyricstranslate

0     Priznajem, griješio sam, baš kao i svi\n1\nZna...
1                                                   NaN
2     Da pospremim\nU srcu lom\nMrka planina\nDala m...
3                                                   NaN
4     Uzmi me sa sobom i povedi me\n1\nI povedi me, ...
5                                                   NaN
6     Dolje u luci barka me čeka\n1\nNeka pričeka je...
7                                                   NaN
8     U kafiću stojim, hladnu coca-colu pijem\nProla...
9                                                   NaN
10    Budi me prije jutra\njavlja mi, stižem sutra\n...
11    (Refren)\n1\nLaž, stoljećima caruje\nsvoje slu...
12                                                  NaN
13    Popio sam noćas metar vina\nKaže doktor da mi ...
14                                                  NaN
15    Ako želiš, zamisli da pjevam ti o nečem drugom...
16                                                  NaN
17                                              

In [87]:
df.to_csv(f"data/{artist_name}_tekstovinet_lyricstranslate.csv", index=False)
